In [ ]:
!pip install ultralytics

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt  
from keras_tuner import RandomSearch
from tensorflow.keras.models import load_model
import glob as gb
import pandas as pd
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from keras_tuner.tuners import RandomSearch
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Input 
from tensorflow.keras.optimizers import Adam, Nadam, Adamax,RMSprop 
from keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from ultralytics import YOLO


In [3]:
yolo_model = YOLO('/kaggle/input/cvm-yolo-detection/weights/best.pt')
img_size = 224

In [ ]:
def load_images_with_yolo(folder_path):
    images = []
    labels = []    
    class_names = os.listdir(folder_path)
    for class_index, class_name in enumerate(class_names):
        class_folder = os.path.join(folder_path, class_name)
        if not os.path.isdir(class_folder):
            continue
        for filename in tqdm(os.listdir(class_folder)):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(class_folder, filename)
                image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                results = yolo_model(image, verbose=False)
                boxes = results[0].boxes.xyxy
                if len(boxes) == 0:
                    continue
                x1, y1, x2, y2 = map(int, boxes[0][:4])
                h, w, _ = image.shape
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w, x2), min(h, y2)
                if x1 >= x2 or y1 >= y2:
                    continue
                cropped = image[y1:y2, x1:x2]
                if cropped.size > 0:
                    cropped = cv2.resize(cropped, (img_size, img_size))
                    cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
                    images.append(cropped)
                    labels.append(class_index)
    return images, labels
#training_set
train_dataset_path = '/kaggle/input/cvmdataset/Cephalometric_Dataset/training_set'
train_images, train_labels = load_images_with_yolo(train_dataset_path)
#testing_set
test_dataset_path = '/kaggle/input/cvmdataset/Cephalometric_Dataset/test_set'
test_images, test_labels = load_images_with_yolo(test_dataset_path)

In [ ]:
random_index = np.random.randint(0, len(train_images))
random_image = train_images[random_index]

plt.imshow(random_image, cmap='gray')
plt.title(f"Random image from class: {train_labels[random_index]}")
plt.axis('off') 
plt.show()


In [ ]:
x_train = np.array(train_images)
y_train = np.array(train_labels)
print("train_images.shape = ", x_train.shape)
print("train_labels.shape = ", y_train.shape)

In [ ]:
x_test = np.array(test_images)
y_test = np.array(test_labels)
print("test_images.shape = ", x_test.shape)
print("test_labels.shape = ", y_test.shape)

In [16]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=42)


In [ ]:
print(f"Training set's total images: {len(y_train)}")
print(f"Validation set's total images: {len(y_val)}")
print(f"Testing set's total images: {len(y_test)}")

In [18]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [19]:
x_train = x_train/255.0
x_val = x_val/255.0
x_test = x_test/255.0

In [ ]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")


In [18]:
from tensorflow.keras.applications import MobileNetV2
def build_model(hp):
    layer_1 = hp.Int(name="layer_1", min_value=32, max_value=512, step=32)
    layer_2 = hp.Int(name="layer_2", min_value=32, max_value=512, step=32)  
    dropout_1 = hp.Float(name="dropout_1", min_value=0.0, max_value=0.5, step=0.1)
    dropout_2 = hp.Float(name="dropout_2", min_value=0.0, max_value=0.5, step=0.1)
    base_model = MobileNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights="imagenet")
    base_model.trainable = False  
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(), 
        Dense(layer_1, activation='relu'),
        Dropout(dropout_1),
        Dense(layer_2, activation='relu'),
        Dropout(dropout_2),
        Dense(6, activation='softmax') ])
    optimizer_type = hp.Choice(name="optimizer_type", values=['nadam', 'rmsprop', 'adam'])
    learning_rate = hp.Choice('learning_rate', values=[1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    if optimizer_type == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_type == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_type == 'nadam':
        optimizer = Nadam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [19]:
tuner = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 10,
    directory = 'CNN',
    project_name ='CVM_Classification'
)

In [ ]:
stop_early = EarlyStopping(monitor='val_loss', patience = 10)
tuner.search(x_train, y_train, epochs=200,callbacks=[stop_early] ,batch_size=8, validation_data=(x_val, y_val))

In [ ]:
model = tuner.hypermodel.build(best_hps)
model.summary()

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print(best_hps.values)

In [ ]:
stop_early = EarlyStopping(monitor='loss', patience = 10)
history = model.fit(x_train, y_train, epochs=500,callbacks=[stop_early] ,batch_size=8, validation_data=(x_val, y_val))
history

In [ ]:

plt.figure(figsize=(14, 5)) 

# Plotting accuracy
plt.subplot(1, 2, 1)  
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()

# Plotting loss
plt.subplot(1, 2, 2)  # 1 row, 2 columns, plot 2
loss = history.history["loss"]
val_loss = history.history["val_loss"]
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()

plt.show()


In [ ]:
val_loss, val_acc = model.evaluate(x_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

In [ ]:
loss, accuracy = model.evaluate(x_test,y_test)
print("Accuracy:", accuracy * 100,"%")
print("Loss:", loss)

In [ ]:
model.save("cvm_model_accuracy83.h5")

In [ ]:

model = load_model("/kaggle/working/finalmodel_accuracy83.h5")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
x_test = np.expand_dims(x_test, axis=-1)  
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

print(classification_report(y_true, y_pred_classes))
